In [1]:
# Install required packages
!pip install -q yfinance pandas numpy plotly scikit-learn ipywidgets
!pip install -q requests nest_asyncio aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import json
import warnings
warnings.filterwarnings('ignore')

from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import aiohttp
import requests
from collections import defaultdict
import re

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, clear_output, HTML, Javascript
import ipywidgets as widgets

In [3]:
# 1. OPENROUTER.AI CLIENT IMPLEMENTATION
class OpenRouterClient:
    """Client for OpenRouter.ai API"""

    def __init__(self, api_key: str):
        if not api_key:
            raise ValueError("❌ API key is required for OpenRouter.ai")

        self.api_key = api_key
        self.base_url = "https://openrouter.ai/api/v1"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com",  # Required by OpenRouter
            "X-Title": "LLM Investment Council"  # Optional but good practice
        }

        # Available models on OpenRouter
        self.available_models = {
            "Meta: Llama 3 70B": {
                "id": "meta-llama/llama-3-70b-instruct",
                "description": "Powerful 70B parameter model, excellent for complex analysis",
                "context_window": 8192,
                "pricing": {"input": 0.59, "output": 0.79}  # $ per 1M tokens
            },
            "Google: Gemini Pro 1.5": {
                "id": "google/gemini-pro-1.5",
                "description": "Google's advanced model, great for reasoning",
                "context_window": 1000000,
                "pricing": {"input": 0.125, "output": 0.375}
            },
            "Anthropic: Claude 3 Opus": {
                "id": "anthropic/claude-3-opus",
                "description": "Most capable Claude model, excellent for investment analysis",
                "context_window": 200000,
                "pricing": {"input": 15.0, "output": 75.0}
            },
            "Anthropic: Claude 3 Sonnet": {
                "id": "anthropic/claude-3-sonnet",
                "description": "Balanced Claude model, cost-effective",
                "context_window": 200000,
                "pricing": {"input": 3.0, "output": 15.0}
            },
            "Anthropic: Claude 3 Haiku": {
                "id": "anthropic/claude-3-haiku",
                "description": "Fastest Claude model, good for quick analysis",
                "context_window": 200000,
                "pricing": {"input": 0.25, "output": 1.25}
            },
            "Mistral AI: Mixtral 8x22B": {
                "id": "mistralai/mixtral-8x22b-instruct",
                "description": "Mistral's powerful model, great for technical analysis",
                "context_window": 65536,
                "pricing": {"input": 0.65, "output": 0.65}
            },
            "OpenAI: GPT-4 Turbo": {
                "id": "openai/gpt-4-turbo",
                "description": "OpenAI's latest GPT-4, excellent all-around",
                "context_window": 128000,
                "pricing": {"input": 10.0, "output": 30.0}
            },
            "OpenAI: GPT-3.5 Turbo": {
                "id": "openai/gpt-3.5-turbo",
                "description": "Fast and cost-effective, good for basic analysis",
                "context_window": 16385,
                "pricing": {"input": 0.5, "output": 1.5}
            },
            "Cohere: Command R+": {
                "id": "cohere/command-r-plus",
                "description": "Excellent for business and financial analysis",
                "context_window": 128000,
                "pricing": {"input": 3.0, "output": 15.0}
            }
        }

    async def query_model(self, model_name: str, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Dict:
        """Query model using OpenRouter API"""

        if model_name not in self.available_models:
            raise ValueError(f"Model '{model_name}' not available. Choose from: {list(self.available_models.keys())}")

        model_id = self.available_models[model_name]["id"]
        url = f"{self.base_url}/chat/completions"

        payload = {
            "model": model_id,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an expert investment advisor. Provide detailed, structured analysis."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": 0.95,
            "frequency_penalty": 0,
            "presence_penalty": 0
        }

        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(
                    url,
                    headers=self.headers,
                    json=payload,
                    timeout=aiohttp.ClientTimeout(total=120)
                ) as response:

                    if response.status == 200:
                        result = await response.json()
                        return {
                            "success": True,
                            "response": self._parse_response(result),
                            "model": model_name,
                            "temperature": temperature,
                            "usage": result.get("usage", {}),
                            "raw": result
                        }
                    else:
                        error_text = await response.text()
                        return {
                            "success": False,
                            "error": f"API Error {response.status}: {error_text[:200]}",
                            "model": model_name
                        }

        except asyncio.TimeoutError:
            return {
                "success": False,
                "error": "Request timed out (120s). Try with shorter prompt.",
                "model": model_name
            }
        except Exception as e:
            return {
                "success": False,
                "error": f"Connection error: {str(e)}",
                "model": model_name
            }

    def _parse_response(self, result) -> str:
        """Parse API response"""
        try:
            if "choices" in result and len(result["choices"]) > 0:
                choice = result["choices"][0]
                if "message" in choice and "content" in choice["message"]:
                    return choice["message"]["content"]
                elif "text" in choice:
                    return choice["text"]

            # Fallback: try to extract any text
            import json
            return json.dumps(result, indent=2)[:1000]

        except Exception as e:
            return f"Error parsing response: {str(e)}"

    def test_connection(self) -> bool:
        """Test if API key is valid"""
        try:
            # Try to get available models endpoint
            response = requests.get(
                f"{self.base_url}/models",
                headers={"Authorization": f"Bearer {self.api_key}"},
                timeout=10
            )
            return response.status_code == 200
        except:
            # Try a different endpoint
            try:
                response = requests.get(
                    "https://openrouter.ai/api/v1/auth/key",
                    headers={"Authorization": f"Bearer {self.api_key}"},
                    timeout=10
                )
                return response.status_code == 200
            except:
                return False

    def get_model_info(self, model_name: str) -> Dict:
        """Get detailed information about a model"""
        if model_name in self.available_models:
            return self.available_models[model_name]
        return {}

    def estimate_cost(self, model_name: str, input_tokens: int, output_tokens: int) -> float:
        """Estimate cost for API call"""
        if model_name in self.available_models:
            pricing = self.available_models[model_name]["pricing"]
            input_cost = (input_tokens / 1_000_000) * pricing["input"]
            output_cost = (output_tokens / 1_000_000) * pricing["output"]
            return round(input_cost + output_cost, 6)
        return 0.0

In [4]:
# 2. ENHANCED COUNCIL MEMBER WITH OPENROUTER
@dataclass
class OpenRouterCouncilMember:
    """Council member configured for OpenRouter"""
    name: str
    role: str
    model_name: str
    temperature: float = 0.7
    voting_weight: float = 1.0
    expertise: str = ""
    max_tokens: int = 1000

    def get_detailed_prompt(self, question: str) -> str:
        """Create detailed prompt for investment analysis"""

        role_contexts = {
            "Conservative": {
                "style": "cautious and risk-averse",
                "focus": ["capital preservation", "downside risk", "safety", "risk management"],
                "bias": "Conservative - prioritize safety over returns"
            },
            "Growth": {
                "style": "aggressive and forward-looking",
                "focus": ["growth potential", "innovation", "market trends", "future potential"],
                "bias": "Growth-oriented - accept higher risk for higher returns"
            },
            "Quantitative": {
                "style": "data-driven and statistical",
                "focus": ["metrics", "statistics", "models", "data analysis", "probability"],
                "bias": "Data-driven - base decisions on quantitative analysis"
            },
            "Technical": {
                "style": "chart-focused and pattern-based",
                "focus": ["price action", "charts", "indicators", "trends", "support/resistance"],
                "bias": "Technical - follow price action and chart patterns"
            },
            "Value": {
                "style": "fundamental and valuation-focused",
                "focus": ["intrinsic value", "margin of safety", "fundamentals", "valuation"],
                "bias": "Value-focused - look for undervalued opportunities"
            },
            "Macro": {
                "style": "big-picture and economic",
                "focus": ["economic cycles", "interest rates", "inflation", "geopolitics"],
                "bias": "Macro-economic - consider broad economic factors"
            },
            "ESG": {
                "style": "sustainable and ethical",
                "focus": ["environmental", "social", "governance", "sustainability", "ethics"],
                "bias": "ESG-focused - prioritize sustainable investments"
            }
        }

        context = role_contexts.get(self.role, role_contexts["Conservative"])

        prompt = f"""You are {self.name}, a {self.role} investment advisor with expertise in {self.expertise if self.expertise else context['focus'][0]}.
Your investment style is {context['style']} and you focus on: {', '.join(context['focus'])}.
Your natural bias is: {context['bias']}.

INVESTMENT QUESTION TO ANALYZE:
{question}

IMPORTANT: You MUST provide your analysis in the following STRUCTURED FORMAT:

1. EXECUTIVE SUMMARY: Brief 2-3 sentence overview of your position

2. KEY ANALYSIS POINTS (list 3-5 specific points):
   - Point 1: [Specific analysis point]
   - Point 2: [Specific analysis point]
   - Point 3: [Specific analysis point]

3. RISK ASSESSMENT:
   - Overall Risk Level: [Low/Medium/High]
   - Key Risks Identified: [List 2-3 specific risks]

4. VALUATION ASSESSMENT (if applicable):
   - Current Valuation: [Overvalued/Fairly Valued/Undervalued]
   - Key Metrics: [List relevant metrics]

5. RECOMMENDATION:
   - Action: [BUY / HOLD / SELL / AVOID]
   - Confidence Level: [1-10, where 10 is highest]
   - Position Size (if buying): [0-100% of portfolio]
   - Time Horizon: [Short-term (1-12 months) / Medium-term (1-3 years) / Long-term (3+ years)]

6. VOTING POSITION: [SUPPORT / OPPOSE / NEUTRAL]

7. FINAL REASONING: Detailed explanation of your analysis (3-5 sentences)

Remember to provide specific, actionable insights based on your {self.role} perspective.
Be thorough and justify your reasoning with concrete points."""

        return prompt

@dataclass
class DetailedMemberResponse:
    """Detailed response from each council member"""
    member_name: str
    role: str
    model_used: str
    temperature: float
    voting_weight: float
    raw_response: str
    confidence: float
    vote: str
    recommendation: str
    risk_assessment: str
    time_horizon: str
    position_size: str
    key_points: List[str]
    executive_summary: str
    valuation: str
    usage_metrics: Dict

    def to_dict(self) -> Dict:
        """Convert to dictionary for display"""
        return {
            "Member": self.member_name,
            "Role": self.role,
            "Model": self.model_used,
            "Temp": f"{self.temperature:.1f}",
            "Weight": f"{self.voting_weight:.1f}",
            "Vote": self.vote,
            "Confidence": f"{self.confidence:.0%}",
            "Recommendation": self.recommendation,
            "Risk": self.risk_assessment,
            "Horizon": self.time_horizon,
            "Position": self.position_size,
            "Summary": self.executive_summary[:50] + "..." if len(self.executive_summary) > 50 else self.executive_summary
        }

In [5]:
# 3. OPENROUTER COUNCIL IMPLEMENTATION
class OpenRouterCouncil:
    """Council using OpenRouter.ai API"""

    def __init__(self, api_key: str):
        if not api_key:
            raise ValueError("❌ OpenRouter API key is required")

        self.api_key = api_key
        self.client = OpenRouterClient(api_key)
        self.members: List[OpenRouterCouncilMember] = []
        self.results_history = []
        self.total_cost_estimate = 0.0

        # Test connection
        if not self.client.test_connection():
            raise ValueError("❌ OpenRouter API key invalid or connection failed")

        print("✅ OpenRouter API connection successful!")
        print(f"📊 Available models: {len(self.client.available_models)}")

    def add_member(self, name: str, role: str, model_name: str,
                  temperature: float = 0.7, voting_weight: float = 1.0,
                  expertise: str = "", max_tokens: int = 1000):
        """Add council member with customizable parameters"""

        if model_name not in self.client.available_models:
            raise ValueError(f"Model '{model_name}' not available. Choose from: {list(self.client.available_models.keys())}")

        member = OpenRouterCouncilMember(
            name=name,
            role=role,
            model_name=model_name,
            temperature=temperature,
            voting_weight=voting_weight,
            expertise=expertise,
            max_tokens=max_tokens
        )

        self.members.append(member)

        # Get model info for display
        model_info = self.client.get_model_info(model_name)
        pricing = model_info.get("pricing", {})

        print(f"✅ Added {name} as {role}")
        print(f"   Model: {model_name}")
        print(f"   Parameters: Temp={temperature}, Weight={voting_weight}")
        if pricing:
            print(f"   Cost: ${pricing.get('input', 0)}/1M input, ${pricing.get('output', 0)}/1M output")

    def create_custom_council(self, configurations: List[Dict]):
        """Create council from custom configurations"""
        for config in configurations:
            self.add_member(
                name=config['name'],
                role=config['role'],
                model_name=config['model'],
                temperature=config.get('temperature', 0.7),
                voting_weight=config.get('weight', 1.0),
                expertise=config.get('expertise', ''),
                max_tokens=config.get('max_tokens', 1000)
            )

    async def deliberate(self, question: str) -> 'EnhancedCouncilResult':
        """Run detailed council deliberation"""

        print(f"\n{'='*80}")
        print("🏛️ OPENROUTER COUNCIL DELIBERATION")
        print(f"{'='*80}")
        print(f"\n📋 Question: {question}")
        print(f"👥 {len(self.members)} members deliberating...")
        print(f"⚙️ Member Configurations:")

        for member in self.members:
            model_info = self.client.get_model_info(member.model_name)
            print(f"  • {member.name} ({member.role}):")
            print(f"    Model: {member.model_name}")
            print(f"    Temp: {member.temperature}, Weight: {member.voting_weight}")
            if model_info.get('pricing'):
                print(f"    Cost: ${model_info['pricing']['input']}/1M in, ${model_info['pricing']['output']}/1M out")

        # Get detailed responses from all members
        responses = []
        print(f"\n🔄 Gathering responses from OpenRouter models...")

        for member in self.members:
            print(f"\n  ⏳ {member.name} ({member.role}) analyzing with {member.model_name}...")
            response = await self._get_detailed_response(member, question)
            responses.append(response)

            # Update cost estimate
            if response.usage_metrics:
                input_tokens = response.usage_metrics.get("prompt_tokens", 0)
                output_tokens = response.usage_metrics.get("completion_tokens", 0)
                cost = self.client.estimate_cost(member.model_name, input_tokens, output_tokens)
                self.total_cost_estimate += cost

            vote_emoji = "✅" if response.vote == "SUPPORT" else "❌" if response.vote == "OPPOSE" else "➡️"
            print(f"    {vote_emoji} Done! Vote: {response.vote}, Confidence: {response.confidence:.0%}")

        # Calculate voting results
        voting_summary = self._calculate_detailed_voting(responses)

        # Generate consensus
        consensus = self._determine_enhanced_consensus(voting_summary)

        # Create comparison matrix
        comparison_matrix = self._create_comparison_matrix(responses)

        # Extract insights
        insights = self._extract_insights(responses)

        # Generate final recommendation
        final_recommendation = self._generate_detailed_recommendation(responses, consensus, voting_summary)

        # Create result object
        result = EnhancedCouncilResult(
            question=question,
            timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            member_responses=responses,
            consensus_decision=consensus,
            voting_summary=voting_summary,
            final_recommendation=final_recommendation,
            comparison_matrix=comparison_matrix,
            insights=insights,
            total_cost=self.total_cost_estimate
        )

        self.results_history.append(result)
        return result

    async def _get_detailed_response(self, member: OpenRouterCouncilMember, question: str) -> DetailedMemberResponse:
        """Get detailed response from member using OpenRouter"""

        prompt = member.get_detailed_prompt(question)

        try:
            # Get response from OpenRouter API
            api_result = await self.client.query_model(
                model_name=member.model_name,
                prompt=prompt,
                temperature=member.temperature,
                max_tokens=member.max_tokens
            )

            if not api_result["success"]:
                raise Exception(f"OpenRouter API Error: {api_result.get('error', 'Unknown error')}")

            raw_response = api_result["response"]

            # Parse detailed response
            parsed = self._parse_structured_response(raw_response, member)

            return DetailedMemberResponse(
                member_name=member.name,
                role=member.role,
                model_used=member.model_name,
                temperature=member.temperature,
                voting_weight=member.voting_weight,
                raw_response=raw_response,
                confidence=parsed["confidence"],
                vote=parsed["vote"],
                recommendation=parsed["recommendation"],
                risk_assessment=parsed["risk_assessment"],
                time_horizon=parsed["time_horizon"],
                position_size=parsed["position_size"],
                key_points=parsed["key_points"],
                executive_summary=parsed["executive_summary"],
                valuation=parsed["valuation"],
                usage_metrics=api_result.get("usage", {})
            )

        except Exception as e:
            print(f"    ❌ Error: {str(e)[:100]}")
            # Return error response
            return DetailedMemberResponse(
                member_name=member.name,
                role=member.role,
                model_used=member.model_name,
                temperature=member.temperature,
                voting_weight=member.voting_weight,
                raw_response=f"Error: {str(e)}",
                confidence=0.3,
                vote="NEUTRAL",
                recommendation="Unable to provide recommendation",
                risk_assessment="Unknown",
                time_horizon="N/A",
                position_size="N/A",
                key_points=["Analysis failed due to API error"],
                executive_summary="Analysis failed",
                valuation="N/A",
                usage_metrics={}
            )

    def _parse_structured_response(self, response: str, member: OpenRouterCouncilMember) -> Dict:
        """Parse structured response into components"""

        parsed = {
            "confidence": 0.5,
            "vote": "NEUTRAL",
            "recommendation": "HOLD",
            "risk_assessment": "Medium",
            "time_horizon": "Medium-term",
            "position_size": "Not specified",
            "key_points": [],
            "executive_summary": "",
            "valuation": "Not specified"
        }

        # Try to extract structured sections
        sections = {
            "EXECUTIVE SUMMARY": "executive_summary",
            "KEY ANALYSIS POINTS": "key_points",
            "RISK ASSESSMENT": "risk_assessment",
            "VALUATION ASSESSMENT": "valuation",
            "RECOMMENDATION": "recommendation",
            "VOTING POSITION": "vote",
            "FINAL REASONING": "executive_summary"  # Fallback
        }

        current_section = None
        content_lines = []

        for line in response.split('\n'):
            line = line.strip()

            # Check if this line starts a new section
            section_found = False
            for section_name in sections.keys():
                if line.upper().startswith(section_name) or re.match(rf'^\d+\.?\s*{section_name}', line.upper()):
                    # Save previous section content
                    if current_section and content_lines:
                        self._save_section_content(parsed, sections[current_section], content_lines)

                    # Start new section
                    current_section = section_name
                    content_lines = []
                    section_found = True
                    break

            if not section_found and current_section:
                content_lines.append(line)

        # Save the last section
        if current_section and content_lines:
            self._save_section_content(parsed, sections[current_section], content_lines)

        # Extract confidence from recommendation section
        confidence_match = re.search(r'Confidence Level:\s*(\d+)', response, re.IGNORECASE)
        if confidence_match:
            try:
                score = int(confidence_match.group(1))
                parsed["confidence"] = min(max(score / 10.0, 0), 1)
            except:
                pass

        # Extract position size
        size_match = re.search(r'Position Size.*?(\d+%)', response, re.IGNORECASE)
        if size_match:
            parsed["position_size"] = size_match.group(1)

        # Extract time horizon
        horizon_match = re.search(r'Time Horizon.*?([A-Za-z]+-term)', response, re.IGNORECASE)
        if horizon_match:
            parsed["time_horizon"] = horizon_match.group(1)

        # Extract vote if not already found
        if parsed["vote"] == "NEUTRAL":
            if "SUPPORT" in response.upper() and "OPPOSE" not in response.upper():
                parsed["vote"] = "SUPPORT"
            elif "OPPOSE" in response.upper() and "SUPPORT" not in response.upper():
                parsed["vote"] = "OPPOSE"

        # Extract recommendation if not already found
        if parsed["recommendation"] == "HOLD":
            for action in ["BUY", "SELL", "AVOID", "HOLD"]:
                if action in response.upper():
                    parsed["recommendation"] = action
                    break

        # Extract risk assessment if not already found
        if parsed["risk_assessment"] == "Medium":
            for risk in ["LOW", "MEDIUM", "HIGH"]:
                if risk in response.upper():
                    parsed["risk_assessment"] = risk.capitalize()
                    break

        # If key points are empty, extract bullet points
        if not parsed["key_points"]:
            for line in response.split('\n'):
                if line.strip().startswith(('-', '•', '*', '→')) or re.match(r'^\s*[a-z]\.', line.strip()):
                    point = re.sub(r'^[•\-\*\→\s\d\.]+', '', line.strip())
                    if point and len(point) > 10:
                        parsed["key_points"].append(point)
                        if len(parsed["key_points"]) >= 5:
                            break

        return parsed

    def _save_section_content(self, parsed: Dict, section_key: str, content_lines: List[str]):
        """Save parsed section content"""
        content = ' '.join([line for line in content_lines if line])

        if section_key == "key_points":
            # Extract bullet points
            points = []
            for line in content_lines:
                if line.strip() and (line.strip().startswith(('-', '•', '*')) or re.match(r'^\s*[a-z]\.', line.strip())):
                    point = re.sub(r'^[•\-\*\→\s\d\.]+', '', line.strip())
                    if point:
                        points.append(point)
            parsed["key_points"] = points[:5]  # Keep top 5 points

        elif section_key == "vote":
            if "SUPPORT" in content.upper():
                parsed["vote"] = "SUPPORT"
            elif "OPPOSE" in content.upper():
                parsed["vote"] = "OPPOSE"

        elif section_key == "recommendation":
            for action in ["BUY", "SELL", "AVOID", "HOLD"]:
                if action in content.upper():
                    parsed["recommendation"] = action
                    break

        elif section_key == "risk_assessment":
            for risk in ["LOW", "MEDIUM", "HIGH"]:
                if risk in content.upper():
                    parsed["risk_assessment"] = risk.capitalize()
                    break

        elif section_key == "executive_summary":
            if not parsed["executive_summary"]:
                parsed["executive_summary"] = content[:200]

        elif section_key == "valuation":
            parsed["valuation"] = content[:100]

    def _calculate_detailed_voting(self, responses: List[DetailedMemberResponse]) -> Dict:
        """Calculate detailed voting results with weights"""

        raw_votes = defaultdict(int)
        weighted_votes = defaultdict(float)
        confidence_sum = defaultdict(float)

        for response in responses:
            vote = response.vote
            weight = response.voting_weight
            confidence = response.confidence

            raw_votes[vote] += 1
            weighted_votes[vote] += confidence * weight
            confidence_sum[vote] += confidence

        # Calculate averages
        avg_confidence = {}
        for vote in confidence_sum:
            if raw_votes[vote] > 0:
                avg_confidence[vote] = confidence_sum[vote] / raw_votes[vote]

        # Determine decision
        total_weight = sum(weighted_votes.values())
        if total_weight > 0:
            support_ratio = weighted_votes.get("SUPPORT", 0) / total_weight
            oppose_ratio = weighted_votes.get("OPPOSE", 0) / total_weight

            if support_ratio >= 0.7:
                decision = "STRONG_SUPPORT"
            elif support_ratio >= 0.6:
                decision = "SUPPORT"
            elif support_ratio > oppose_ratio:
                decision = "LEAN_SUPPORT"
            elif oppose_ratio >= 0.7:
                decision = "STRONG_OPPOSE"
            elif oppose_ratio >= 0.6:
                decision = "OPPOSE"
            elif oppose_ratio > support_ratio:
                decision = "LEAN_OPPOSE"
            else:
                decision = "NEUTRAL"
        else:
            decision = "NO_CONSENSUS"

        return {
            "raw_votes": dict(raw_votes),
            "weighted_votes": dict(weighted_votes),
            "average_confidence": avg_confidence,
            "decision": decision,
            "total_members": len(responses)
        }

    def _determine_enhanced_consensus(self, voting_summary: Dict) -> str:
        """Determine enhanced consensus"""

        decision_map = {
            "STRONG_SUPPORT": "✅ STRONG BUY RECOMMENDATION",
            "SUPPORT": "👍 BUY RECOMMENDATION",
            "LEAN_SUPPORT": "🤔 LEAN BUY",
            "NEUTRAL": "⚖️ HOLD / NEUTRAL",
            "LEAN_OPPOSE": "🤔 LEAN SELL",
            "OPPOSE": "👎 SELL RECOMMENDATION",
            "STRONG_OPPOSE": "❌ STRONG SELL RECOMMENDATION",
            "NO_CONSENSUS": "❓ NO CLEAR CONSENSUS"
        }

        return decision_map.get(voting_summary["decision"], voting_summary["decision"])

    def _create_comparison_matrix(self, responses: List[DetailedMemberResponse]) -> pd.DataFrame:
        """Create comparison matrix of all member responses"""

        data = []
        for response in responses:
            data.append(response.to_dict())

        df = pd.DataFrame(data)
        df.index = range(1, len(df) + 1)

        return df

    def _extract_insights(self, responses: List[DetailedMemberResponse]) -> Dict:
        """Extract insights from responses"""

        insights = {
            "by_role": defaultdict(list),
            "by_vote": defaultdict(list),
            "by_model": defaultdict(list),
            "common_themes": []
        }

        # Group by role
        for response in responses:
            if response.key_points:
                insights["by_role"][response.role].extend(response.key_points)

        # Group by vote
        for response in responses:
            if response.key_points:
                insights["by_vote"][response.vote].extend(response.key_points)

        # Group by model
        for response in responses:
            if response.executive_summary:
                insights["by_model"][response.model_used].append(response.executive_summary)

        # Find common themes
        all_points = []
        for response in responses:
            all_points.extend([p.lower() for p in response.key_points])
            all_points.append(response.executive_summary.lower())

        from collections import Counter
        word_counts = Counter()
        for point in all_points:
            words = re.findall(r'\b[a-z]{4,}\b', point.lower())
            word_counts.update(words)

        # Remove common words
        common_words = {'this', 'that', 'with', 'from', 'have', 'been', 'they', 'what',
                       'when', 'where', 'which', 'would', 'could', 'should', 'about'}
        for word in common_words:
            if word in word_counts:
                del word_counts[word]

        # Get top themes
        top_words = [word for word, count in word_counts.most_common(15) if len(word) > 3]
        insights["common_themes"] = top_words[:5]

        return insights

    def _generate_detailed_recommendation(self, responses: List[DetailedMemberResponse],
                                         consensus: str, voting_summary: Dict) -> str:
        """Generate detailed final recommendation"""

        recommendation = f"""
## 🏛️ OPENROUTER COUNCIL FINAL DECISION REPORT

### 🎯 CONSENSUS: {consensus}

### 📊 VOTING BREAKDOWN:
"""

        # Add voting details
        votes = voting_summary["raw_votes"]
        for vote, count in votes.items():
            if count > 0:
                confidence = voting_summary["average_confidence"].get(vote, 0)
                recommendation += f"\n• **{vote}**: {count} member(s) | Average Confidence: {confidence:.0%}"

        # Add cost summary
        total_cost = sum([getattr(r, 'total_cost', 0) for r in responses if hasattr(r, 'total_cost')])
        recommendation += f"\n\n### 💰 ESTIMATED API COST: ${total_cost:.4f}"

        # Add role-based analysis
        recommendation += f"\n\n### 👥 ROLE-BASED ANALYSIS:"

        roles_summary = {}
        for response in responses:
            role = response.role
            if role not in roles_summary:
                roles_summary[role] = {
                    "votes": {"SUPPORT": 0, "OPPOSE": 0, "NEUTRAL": 0},
                    "avg_confidence": 0,
                    "count": 0,
                    "key_insights": []
                }

            roles_summary[role]["votes"][response.vote] += 1
            roles_summary[role]["avg_confidence"] += response.confidence
            roles_summary[role]["count"] += 1
            if response.key_points:
                roles_summary[role]["key_insights"].extend(response.key_points[:2])

        for role, summary in roles_summary.items():
            if summary["count"] > 0:
                avg_conf = summary["avg_confidence"] / summary["count"]
                dominant_vote = max(summary["votes"].items(), key=lambda x: x[1])
                recommendation += f"\n\n**{role.upper()} PERSPECTIVE**:"
                recommendation += f"\n• Vote: {dominant_vote[0]} ({dominant_vote[1]}/{summary['count']} members)"
                recommendation += f"\n• Average Confidence: {avg_conf:.0%}"
                if summary["key_insights"]:
                    recommendation += f"\n• Key Insights: {', '.join(summary['key_insights'][:2])}"

        # Add model performance
        recommendation += f"\n\n### 🤖 MODEL PERFORMANCE:"

        models_summary = {}
        for response in responses:
            model = response.model_used
            if model not in models_summary:
                models_summary[model] = {"count": 0, "avg_confidence": 0, "votes": []}

            models_summary[model]["count"] += 1
            models_summary[model]["avg_confidence"] += response.confidence
            models_summary[model]["votes"].append(response.vote)

        for model, summary in models_summary.items():
            if summary["count"] > 0:
                avg_conf = summary["avg_confidence"] / summary["count"]
                support_pct = (summary["votes"].count("SUPPORT") / len(summary["votes"])) * 100
                recommendation += f"\n• **{model}**: {support_pct:.0f}% supportive | Avg Confidence: {avg_conf:.0%}"

        # Add action plan based on consensus
        recommendation += f"\n\n### 📈 RECOMMENDED ACTION PLAN:"

        if "BUY" in consensus:
            recommendation += f"""
1. **IMMEDIATE ACTION**: Consider initiating a position
2. **POSITION SIZING**: Based on council average: {self._calculate_average_position_size(responses)}
3. **ENTRY STRATEGY**: Dollar-cost average over 2-4 weeks
4. **RISK MANAGEMENT**: Set stop-loss at 15-20% below entry
5. **PROFIT TARGETS**: 20-30% upside based on analysis"""
        elif "SELL" in consensus:
            recommendation += f"""
1. **IMMEDIATE ACTION**: Consider reducing exposure
2. **EXIT STRATEGY**: Scale out over 1-2 weeks
3. **HEDGING**: Consider protective options if holding
4. **CAPITAL PRESERVATION**: Move to cash or defensive assets
5. **RE-ENTRY CRITERIA**: Wait for specific conditions"""
        else:
            recommendation += f"""
1. **IMMEDIATE ACTION**: Maintain current position
2. **MONITORING**: Watch key levels identified by council
3. **ADDITION CRITERIA**: Add only if specific conditions met
4. **REDUCTION TRIGGERS**: Reduce if specific risks materialize
5. **REVIEW TIMELINE**: Re-evaluate in 4-8 weeks"""

        # Add key risks identified
        recommendation += f"\n\n### ⚠️ KEY RISKS IDENTIFIED:"

        high_risks = []
        for response in responses:
            if response.risk_assessment == "High" and response.key_points:
                high_risks.extend(response.key_points[:2])

        if high_risks:
            unique_risks = list(set(high_risks))[:3]
            for i, risk in enumerate(unique_risks, 1):
                recommendation += f"\n{i}. {risk}"
        else:
            recommendation += "\n• No specific high risks identified by majority of council members"

        # Add next steps
        recommendation += f"\n\n### 🚀 NEXT STEPS:"
        recommendation += f"""
1. Review detailed analysis from each council member
2. Consider the weighted voting (confidence × weight)
3. Monitor key metrics mentioned in analyses
4. Set calendar reminder for review based on time horizons
5. Document decision rationale for future reference"""

        return recommendation

    def _calculate_average_position_size(self, responses: List[DetailedMemberResponse]) -> str:
        """Calculate average position size from recommendations"""
        sizes = []
        for response in responses:
            if response.position_size and '%' in response.position_size:
                try:
                    size = float(response.position_size.replace('%', ''))
                    sizes.append(size)
                except:
                    pass

        if sizes:
            avg_size = np.mean(sizes)
            return f"{avg_size:.1f}% of portfolio"
        return "5-10% suggested (no specific consensus)"

In [6]:
# 4. ENHANCED COUNCIL RESULT CLASS
@dataclass
class EnhancedCouncilResult:
    """Enhanced council deliberation result"""
    question: str
    timestamp: str
    member_responses: List[DetailedMemberResponse]
    consensus_decision: str
    voting_summary: Dict
    final_recommendation: str
    comparison_matrix: pd.DataFrame
    insights: Dict[str, List[str]]
    total_cost: float = 0.0

    def display_comprehensive(self):
        """Display comprehensive results"""
        print("\n" + "="*80)
        print("🏛️ OPENROUTER INVESTMENT COUNCIL - COMPREHENSIVE REPORT")
        print("="*80)

        print(f"\n📋 QUESTION: {self.question}")
        print(f"⏰ TIMESTAMP: {self.timestamp}")
        print(f"💰 ESTIMATED COST: ${self.total_cost:.4f}")

        print(f"\n🎯 FINAL CONSENSUS: {self.consensus_decision}")
        print(f"📊 VOTING RESULT: {self.voting_summary['decision']}")

        print(f"\n{'='*80}")
        print("📈 DETAILED MEMBER ANALYSIS")
        print("="*80)

        for i, response in enumerate(self.member_responses, 1):
            print(f"\n{i}. {response.member_name} ({response.role})")
            print(f"   Model: {response.model_used} | Temp: {response.temperature} | Weight: {response.voting_weight}")
            print(f"   Vote: {response.vote} | Confidence: {response.confidence:.0%}")
            print(f"   Recommendation: {response.recommendation}")
            print(f"   Risk Assessment: {response.risk_assessment}")
            print(f"   Time Horizon: {response.time_horizon}")
            print(f"   Position Size: {response.position_size}")
            print(f"   Valuation: {response.valuation}")

            print(f"\n   Executive Summary:")
            print(f"   {response.executive_summary}")

            print(f"\n   Key Points:")
            for point in response.key_points[:3]:
                print(f"     • {point}")

            print(f"\n   Full Analysis (excerpt):")
            print(f"   {response.raw_response[:300]}...")
            print("-" * 60)

        print(f"\n{'='*80}")
        print("📊 COMPARISON MATRIX")
        print("="*80)
        print(self.comparison_matrix.to_string())

        print(f"\n{'='*80}")
        print("💡 KEY INSIGHTS")
        print("="*80)

        print("\n🔍 By Role Perspective:")
        for role, points in self.insights.get('by_role', {}).items():
            if points:
                print(f"\n{role}:")
                for point in points[:2]:
                    print(f"  • {point}")

        print("\n🔍 By Voting Group:")
        for vote, points in self.insights.get('by_vote', {}).items():
            if points:
                print(f"\n{vote} Voters:")
                for point in points[:2]:
                    print(f"  • {point}")

        print("\n🔍 Common Themes:")
        for theme in self.insights.get('common_themes', [])[:5]:
            print(f"  • {theme.title()}")

        print(f"\n{'='*80}")
        print("🎯 FINAL RECOMMENDATION")
        print("="*80)
        print(f"\n{self.final_recommendation}")

In [7]:
# 5. COMPREHENSIVE INTERFACE FOR OPENROUTER
class OpenRouterCouncilInterface:
    """Comprehensive interface for OpenRouter Council"""

    def __init__(self):
        self.api_key = None
        self.council = None
        self.member_configs = []

        self._create_widgets()

    def _create_widgets(self):
        """Create comprehensive widgets for OpenRouter"""

        # API Key Section
        self.api_input = widgets.Password(
            value="",
            placeholder='Enter your OpenRouter.ai API key',
            description='API Key:',
            layout=widgets.Layout(width='500px'),
            style={'description_width': 'initial'}
        )

        self.test_api_btn = widgets.Button(
            description='🔍 Test API',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        self.test_api_btn.on_click(self._test_api)

        # Member Configuration
        self.member_widgets = []

        # Available models (will be populated)
        self.model_options = ["Google: Gemini Pro 1.5", "Meta: Llama 3 70B",
                             "Anthropic: Claude 3 Sonnet", "Mistral AI: Mixtral 8x22B",
                             "OpenAI: GPT-4 Turbo", "OpenAI: GPT-3.5 Turbo"]

        # Role options
        self.role_options = ["Conservative", "Growth", "Quantitative",
                            "Technical", "Value", "Macro", "ESG"]

        # Create 5 member configuration sections
        for i in range(5):
            member_box = widgets.VBox([
                widgets.HTML(f"<h4>Council Member {i+1}</h4>"),

                # Name
                widgets.Text(
                    value=f"Member {i+1}",
                    description='Name:',
                    layout=widgets.Layout(width='300px')
                ),

                # Role
                widgets.Dropdown(
                    options=self.role_options,
                    value=self.role_options[i % len(self.role_options)],
                    description='Role:',
                    layout=widgets.Layout(width='300px')
                ),

                # Model Selection
                widgets.Dropdown(
                    options=self.model_options,
                    value=self.model_options[i % len(self.model_options)],
                    description='Model:',
                    layout=widgets.Layout(width='300px')
                ),

                # Temperature Slider with value display
                widgets.FloatSlider(
                    value=0.7,
                    min=0.1,
                    max=1.0,
                    step=0.1,
                    description='Temperature:',
                    readout=True,
                    readout_format='.1f',
                    layout=widgets.Layout(width='300px')
                ),

                # Voting Weight Slider
                widgets.FloatSlider(
                    value=1.0,
                    min=0.1,
                    max=3.0,
                    step=0.1,
                    description='Weight:',
                    readout=True,
                    readout_format='.1f',
                    layout=widgets.Layout(width='300px')
                ),

                # Max Tokens
                widgets.IntSlider(
                    value=1000,
                    min=500,
                    max=4000,
                    step=100,
                    description='Max Tokens:',
                    layout=widgets.Layout(width='300px')
                ),

                # Expertise
                widgets.Text(
                    value="",
                    placeholder='Special expertise (optional)...',
                    description='Expertise:',
                    layout=widgets.Layout(width='300px')
                )
            ])

            self.member_widgets.append({
                'container': member_box,
                'name': member_box.children[1],
                'role': member_box.children[2],
                'model': member_box.children[3],
                'temperature': member_box.children[4],
                'weight': member_box.children[5],
                'max_tokens': member_box.children[6],
                'expertise': member_box.children[7]
            })

        # Setup Council Button
        self.setup_btn = widgets.Button(
            description='🏛️ Setup Council',
            button_style='success',
            layout=widgets.Layout(width='150px', height='40px'),
            disabled=True
        )
        self.setup_btn.on_click(self._setup_council)

        # Question Section
        self.question_input = widgets.Textarea(
            value="Should I invest in NVIDIA (NVDA) stock given the current AI boom and high valuation? Consider growth prospects, competition, and valuation metrics.",
            placeholder='Enter your detailed investment question...',
            description='Question:',
            layout=widgets.Layout(width='600px', height='120px')
        )

        # Sample Questions
        self.sample_questions = widgets.Dropdown(
            options=[
                "Should I invest in Apple (AAPL) at current valuation levels of $180+ per share?",
                "Is Bitcoin a good long-term investment and inflation hedge in the current economic environment?",
                "Should I allocate more to growth stocks or value stocks for 2024 portfolio?",
                "Is the AI investment trend sustainable long-term or a bubble about to burst?",
                "Should I rebalance my portfolio towards defensive sectors given current market volatility?",
                "What's the optimal bond/stock allocation for a moderate risk tolerance investor?"
            ],
            value="Should I invest in Apple (AAPL) at current valuation levels of $180+ per share?",
            description='Sample Questions:',
            layout=widgets.Layout(width='500px')
        )
        self.sample_questions.observe(self._on_sample_select, 'value')

        # Action Buttons
        self.ask_btn = widgets.Button(
            description='🤔 Ask Council',
            button_style='primary',
            layout=widgets.Layout(width='150px', height='40px'),
            disabled=True
        )
        self.ask_btn.on_click(self._ask_council)

        self.reset_btn = widgets.Button(
            description='🔄 Reset',
            button_style='warning',
            layout=widgets.Layout(width='120px', height='40px')
        )
        self.reset_btn.on_click(self._reset)

        # Progress Bar
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Progress:',
            bar_style='info',
            layout=widgets.Layout(width='500px')
        )

        # Cost Estimate
        self.cost_label = widgets.HTML(
            value="<div style='padding: 10px; background: #f8f9fa; border-radius: 5px;'>💰 Estimated Cost: $0.00</div>",
            layout=widgets.Layout(width='500px')
        )

        # Output Areas
        self.status_output = widgets.Output()
        self.results_output = widgets.Output()
        self.comparison_output = widgets.Output()
        self.visualization_output = widgets.Output()

    def _test_api(self, b):
        """Test OpenRouter API key"""
        with self.status_output:
            clear_output(wait=True)

            api_key = self.api_input.value.strip()
            if not api_key:
                print("❌ Please enter an OpenRouter API key")
                return

            print("🔍 Testing OpenRouter API connection...")

            try:
                # Test with a simple request
                headers = {
                    "Authorization": f"Bearer {api_key}",
                    "Content-Type": "application/json"
                }

                # Try to get auth info
                response = requests.get(
                    "https://openrouter.ai/api/v1/auth/key",
                    headers=headers,
                    timeout=10
                )

                if response.status_code == 200:
                    print("✅ OpenRouter API key is valid!")
                    print("📊 You can now configure your council members")
                    self.api_key = api_key
                    self.setup_btn.disabled = False

                    # Show available credits if possible
                    try:
                        data = response.json()
                        if 'data' in data and 'credits' in data['data']:
                            credits = data['data']['credits']
                            print(f"💰 Available credits: ${credits:.2f}")
                    except:
                        pass

                else:
                    print(f"❌ API key invalid. Status: {response.status_code}")
                    self.setup_btn.disabled = True
                    self.ask_btn.disabled = True

            except Exception as e:
                print(f"❌ Connection error: {str(e)}")
                print("\n💡 Make sure:")
                print("1. You have a valid OpenRouter API key")
                print("2. You have sufficient credits")
                print("3. Your network allows API calls")
                self.setup_btn.disabled = True
                self.ask_btn.disabled = True

    def _on_sample_select(self, change):
        """Handle sample question selection"""
        self.question_input.value = change['new']

    def _setup_council(self, b):
        """Setup council with configured members"""
        with self.status_output:
            clear_output(wait=True)

            if not self.api_key:
                print("❌ Please enter and test your API key first")
                return

            print("🏛️ Setting up OpenRouter Council...")

            try:
                # Create council
                self.council = OpenRouterCouncil(self.api_key)

                # Add members from configuration
                for i, member_widget in enumerate(self.member_widgets):
                    self.council.add_member(
                        name=member_widget['name'].value,
                        role=member_widget['role'].value,
                        model_name=member_widget['model'].value,
                        temperature=member_widget['temperature'].value,
                        voting_weight=member_widget['weight'].value,
                        expertise=member_widget['expertise'].value,
                        max_tokens=member_widget['max_tokens'].value
                    )

                print(f"✅ Council setup complete with {len(self.council.members)} members")
                print("\n📋 Council Configuration Summary:")
                for member in self.council.members:
                    print(f"  • {member.name} ({member.role})")
                    print(f"    Model: {member.model_name}")
                    print(f"    Temp: {member.temperature}, Weight: {member.voting_weight}")
                    print(f"    Max Tokens: {member.max_tokens}")
                    if member.expertise:
                        print(f"    Expertise: {member.expertise}")
                    print()

                self.ask_btn.disabled = False
                print("💡 Council is ready! Enter your question and click 'Ask Council'")

            except Exception as e:
                print(f"❌ Error setting up council: {str(e)}")
                self.ask_btn.disabled = True

    async def _ask_async(self, question: str):
        """Async function to ask council"""
        if not self.council:
            return None

        # Update progress
        self.progress.value = 10

        # Run deliberation
        result = await self.council.deliberate(question)

        # Update progress
        self.progress.value = 100

        # Update cost estimate
        self.cost_label.value = f"<div style='padding: 10px; background: #e8f5e8; border-radius: 5px;'>💰 Estimated Cost: ${result.total_cost:.4f}</div>"

        return result

    def _ask_council(self, b):
        """Ask council question"""
        with self.results_output:
            clear_output(wait=True)

        with self.comparison_output:
            clear_output(wait=True)

        with self.visualization_output:
            clear_output(wait=True)

        with self.status_output:
            clear_output(wait=True)

            if not self.council:
                print("❌ Please setup council first!")
                return

            question = self.question_input.value.strip()
            if not question:
                print("❌ Please enter a question")
                return

            print("🤔 Council is deliberating via OpenRouter...")
            print("⏳ This may take 2-3 minutes for detailed analysis...")
            print("💰 Each member's analysis incurs API costs")
            self.progress.value = 20

            try:
                # Run async deliberation
                result = asyncio.run(self._ask_async(question))

                if result:
                    # Display comprehensive results
                    with self.results_output:
                        result.display_comprehensive()

                    # Display comparison matrix
                    with self.comparison_output:
                        self._display_comparison(result)

                    # Display visualizations
                    with self.visualization_output:
                        self._display_visualizations(result)

                    print("✅ Deliberation complete!")
                    print(f"💰 Total estimated cost: ${result.total_cost:.4f}")

            except Exception as e:
                print(f"❌ Error during deliberation: {str(e)}")
                print("\n💡 Common solutions:")
                print("1. Check your OpenRouter API key has sufficient credits")
                print("2. Try a simpler/shorter question")
                print("3. Reduce max_tokens for each member")
                print("4. Use cheaper models (Claude 3 Haiku, GPT-3.5 Turbo)")

            self.progress.value = 0

    def _display_comparison(self, result: EnhancedCouncilResult):
        """Display comparison view"""

        print("\n" + "="*80)
        print("📊 SIDE-BY-SIDE COMPARISON")
        print("="*80)

        # Display the comparison matrix
        display(result.comparison_matrix)

        # Add voting analysis
        print("\n📈 VOTING ANALYSIS:")
        votes = result.voting_summary["raw_votes"]
        for vote, count in votes.items():
            percentage = (count / len(result.member_responses)) * 100
            print(f"  {vote}: {count} votes ({percentage:.1f}%)")

        # Add confidence analysis
        print("\n💪 CONFIDENCE ANALYSIS:")
        confidences = [r.confidence for r in result.member_responses]
        print(f"  Average Confidence: {np.mean(confidences):.1%}")
        print(f"  Highest: {max(confidences):.1%} ({result.member_responses[confidences.index(max(confidences))].member_name})")
        print(f"  Lowest: {min(confidences):.1%} ({result.member_responses[confidences.index(min(confidences))].member_name})")

        # Add cost breakdown
        print("\n💰 COST BREAKDOWN BY MEMBER:")
        for response in result.member_responses:
            if hasattr(response, 'usage_metrics') and response.usage_metrics:
                tokens = response.usage_metrics.get('total_tokens', 0)
                # Simple estimate (0.000001 per token as rough average)
                cost_est = tokens * 0.000001
                print(f"  {response.member_name}: ~{tokens} tokens (${cost_est:.4f})")

    def _display_visualizations(self, result: EnhancedCouncilResult):
        """Display visualizations"""

        try:
            # Create subplots
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=('Voting Distribution', 'Confidence by Model',
                              'Temperature vs Weight', 'Risk Assessment Distribution'),
                specs=[[{'type': 'pie'}, {'type': 'bar'}],
                       [{'type': 'scatter'}, {'type': 'bar'}]],
                vertical_spacing=0.15,
                horizontal_spacing=0.15
            )

            # 1. Voting Pie Chart
            votes = result.voting_summary["raw_votes"]
            fig.add_trace(
                go.Pie(
                    labels=list(votes.keys()),
                    values=list(votes.values()),
                    hole=0.3,
                    marker=dict(colors=['#28a745', '#dc3545', '#ffc107']),
                    name="Votes",
                    textinfo='label+percent'
                ),
                row=1, col=1
            )

            # 2. Confidence by Model (Bar Chart)
            model_confidences = {}
            for response in result.member_responses:
                model = response.model_used
                if model not in model_confidences:
                    model_confidences[model] = []
                model_confidences[model].append(response.confidence)

            avg_confidences = {model: np.mean(vals) for model, vals in model_confidences.items()}

            fig.add_trace(
                go.Bar(
                    x=list(avg_confidences.keys()),
                    y=list(avg_confidences.values()),
                    marker_color='#007bff',
                    name="Avg Confidence",
                    text=[f'{v:.1%}' for v in avg_confidences.values()],
                    textposition='auto'
                ),
                row=1, col=2
            )

            # 3. Temperature vs Weight (Scatter)
            temps = [r.temperature for r in result.member_responses]
            weights = [r.voting_weight for r in result.member_responses]
            names = [r.member_name for r in result.member_responses]
            votes_colors = ['#28a745' if r.vote == 'SUPPORT' else
                          '#dc3545' if r.vote == 'OPPOSE' else '#ffc107'
                          for r in result.member_responses]

            fig.add_trace(
                go.Scatter(
                    x=temps,
                    y=weights,
                    mode='markers+text',
                    marker=dict(size=20, color=votes_colors, opacity=0.8),
                    text=names,
                    textposition="top center",
                    name="Members",
                    hovertext=[f"{r.member_name}: Temp={r.temperature}, Weight={r.voting_weight}, Vote={r.vote}"
                              for r in result.member_responses]
                ),
                row=2, col=1
            )

            # 4. Risk Assessment Distribution
            risk_counts = {}
            for response in result.member_responses:
                risk = response.risk_assessment
                risk_counts[risk] = risk_counts.get(risk, 0) + 1

            fig.add_trace(
                go.Bar(
                    x=list(risk_counts.keys()),
                    y=list(risk_counts.values()),
                    marker_color=['#dc3545', '#ffc107', '#28a745'][:len(risk_counts)],
                    name="Risk Assessment",
                    text=list(risk_counts.values()),
                    textposition='auto'
                ),
                row=2, col=2
            )

            fig.update_layout(
                height=800,
                showlegend=False,
                title_text="OpenRouter Council Analysis Dashboard",
                template="plotly_white",
                title_font_size=20
            )

            # Update axes labels
            fig.update_xaxes(title_text="Temperature", row=2, col=1)
            fig.update_yaxes(title_text="Voting Weight", row=2, col=1)
            fig.update_xaxes(title_text="Risk Level", row=2, col=2)
            fig.update_yaxes(title_text="Count", row=2, col=2)

            fig.show()

        except Exception as e:
            print(f"Visualization error: {str(e)}")
            print("\nDisplaying simple analysis instead...")

            # Simple text analysis
            print("\n📊 SIMPLE ANALYSIS:")

            print("\nVoting Summary:")
            for vote, count in result.voting_summary["raw_votes"].items():
                if count > 0:
                    bar = "█" * count + " " * (5 - count)
                    print(f"  {vote}: {bar} ({count})")

            print("\nModel Performance:")
            models = {}
            for response in result.member_responses:
                if response.model_used not in models:
                    models[response.model_used] = {"votes": [], "confidence": []}
                models[response.model_used]["votes"].append(response.vote)
                models[response.model_used]["confidence"].append(response.confidence)

            for model, data in models.items():
                support_pct = (data["votes"].count("SUPPORT") / len(data["votes"])) * 100
                avg_conf = np.mean(data["confidence"])
                print(f"  {model}: {support_pct:.0f}% supportive, {avg_conf:.0%} avg confidence")

    def _reset(self, b):
        """Reset interface"""
        with self.status_output:
            clear_output(wait=True)
        with self.results_output:
            clear_output(wait=True)
        with self.comparison_output:
            clear_output(wait=True)
        with self.visualization_output:
            clear_output(wait=True)

        self.council = None
        self.ask_btn.disabled = True
        self.setup_btn.disabled = True
        self.progress.value = 0
        self.cost_label.value = "<div style='padding: 10px; background: #f8f9fa; border-radius: 5px;'>💰 Estimated Cost: $0.00</div>"

        print("🔄 Interface reset. Please enter API key and setup council again.")

    def display(self):
        """Display the comprehensive interface"""

        display(HTML("""
        <div style="background: linear-gradient(135deg, #1a237e 0%, #283593 100%);
                    padding: 25px; border-radius: 15px; margin-bottom: 25px; text-align: center; color: white;">
            <h1 style="color: white; margin: 0; font-size: 2.5em;">🏛️ OPENROUTER INVESTMENT COUNCIL</h1>
            <p style="color: white; opacity: 0.9; font-size: 1.2em; margin: 10px 0;">
                5 AI Models • OpenRouter.ai • Professional Analysis
            </p>
            <p style="color: white; opacity: 0.8; font-size: 0.9em;">
                Uses real API calls - Requires OpenRouter credits
            </p>
        </div>
        """))

        # API Section
        display(HTML("<h2>🔑 Step 1: OpenRouter API Configuration</h2>"))
        display(HTML("""
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;">
            <p><strong>Requirements:</strong></p>
            <ol>
                <li>Get OpenRouter API key from: <a href="https://openrouter.ai/keys" target="_blank">https://openrouter.ai/keys</a></li>
                <li>Add credits to your account (starts with $5 free trial)</li>
                <li>Enter your API key below and test connection</li>
            </ol>
            <p><small>Note: Each council deliberation incurs API costs based on model usage.</small></p>
        </div>
        """))

        display(widgets.HBox([self.api_input, self.test_api_btn]))
        display(self.status_output)

        # Council Configuration
        display(HTML("<h2>👥 Step 2: Configure Council Members</h2>"))
        display(HTML("""
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;">
            <p><strong>Customize each of 5 council members (all using OpenRouter):</strong></p>
            <table style="width: 100%;">
                <tr>
                    <td><strong>Parameter</strong></td>
                    <td><strong>Description</strong></td>
                    <td><strong>Typical Range</strong></td>
                </tr>
                <tr>
                    <td>Temperature</td>
                    <td>Controls creativity/randomness</td>
                    <td>0.1 (conservative) to 1.0 (creative)</td>
                </tr>
                <tr>
                    <td>Voting Weight</td>
                    <td>Influence in final decision</td>
                    <td>0.1 (low) to 3.0 (high influence)</td>
                </tr>
                <tr>
                    <td>Max Tokens</td>
                    <td>Maximum response length</td>
                    <td>500 (brief) to 4000 (detailed)</td>
                </tr>
            </table>
        </div>
        """))

        # Display member configuration in a grid
        member_grid = widgets.GridBox(
            [w['container'] for w in self.member_widgets],
            layout=widgets.Layout(
                grid_template_columns='repeat(2, 1fr)',
                grid_gap='20px',
                width='100%'
            )
        )

        display(member_grid)
        display(self.setup_btn)

        # Question Section
        display(HTML("<h2>💬 Step 3: Ask the Council</h2>"))
        display(self.question_input)
        display(HTML("<p>Or choose a sample question:</p>"))
        display(self.sample_questions)

        # Action Buttons and Progress
        display(widgets.HBox([self.ask_btn, self.reset_btn]))
        display(self.progress)
        display(self.cost_label)

        # Results Sections with Tabs
        display(HTML("<h2>📊 Results</h2>"))

        # Create tabs for different views
        tabs = widgets.Tab(children=[
            self.results_output,
            self.comparison_output,
            self.visualization_output
        ])
        tabs.set_title(0, '📋 Comprehensive Report')
        tabs.set_title(1, '📈 Comparison View')
        tabs.set_title(2, '📊 Visualizations')

        display(tabs)

# Create and display interface
print("🚀 Loading OpenRouter Investment Council Interface...")
print("This interface requires a valid OpenRouter.ai API key with credits")
interface = OpenRouterCouncilInterface()
interface.display()

🚀 Loading OpenRouter Investment Council Interface...
This interface requires a valid OpenRouter.ai API key with credits


Output()

Parameter,Description,Typical Range
Temperature,Controls creativity/randomness,0.1 (conservative) to 1.0 (creative)
Voting Weight,Influence in final decision,0.1 (low) to 3.0 (high influence)
Max Tokens,Maximum response length,500 (brief) to 4000 (detailed)


GridBox(children=(VBox(children=(HTML(value='<h4>Council Member 1</h4>'), Text(value='Member 1', description='…

Button(button_style='success', description='🏛️ Setup Council', disabled=True, layout=Layout(height='40px', wid…

Textarea(value='Should I invest in NVIDIA (NVDA) stock given the current AI boom and high valuation? Consider …

Dropdown(description='Sample Questions:', layout=Layout(width='500px'), options=('Should I invest in Apple (AA…

IntProgress(value=0, bar_style='info', description='Progress:', layout=Layout(width='500px'))

HTML(value="<div style='padding: 10px; background: #f8f9fa; border-radius: 5px;'>💰 Estimated Cost: $0.00</div>…